In [5]:
from PIL import Image
import os

In [88]:


# Open the PNG image
image = Image.open("image_1.3.6.1.4.1.14519.5.2.1.6279.6001.202709423777326615340853838834.png")

# Get the pixel data as a list
pixel_data = list(image.getdata())

# Iterate through the pixel data and modify the values as needed
modified_pixel_data = []
for pixel in pixel_data:
    # Modify the pixel value here (e.g., adjust intensity)
    intensity = pixel  # For grayscale, pixel is a single value
    if intensity <= 33200: 
        intensity = 0
    elif intensity >= 34500:
        intensity = 65535

    modified_pixel_data.append(intensity)

####################################################
#remove this part if the whiteening didn't make it better
def n_colorInRow(pixel_data:list, n:int):
    count = 0
    intervalls = [] #going to be list of lists
    for i, pixel in enumerate(pixel_data):
        if i == 0 and pixel != 0 and pixel_data[1] !=0:
            count +=1
        elif pixel != 0 and pixel_data[i-1] != 0 and pixel_data[i+1] !=0: 
            count+=1
        elif i == len(pixel_data) and pixel_data[i-1] != 0:
            count+=1
        else:
            if count >= n:
                intervalls.append([i-count, i]) 
            count = 0
    return intervalls
        
interval_list = n_colorInRow(modified_pixel_data,25)
for interval in interval_list:
    for i in range(interval[0], interval[1]+1):
        modified_pixel_data[i] = 65535
#####################################################





# Update the image with the modified pixel data
#image.putdata(modified_pixel_data)
image.putdata(modified_pixel_data)

# Save the modified image
image.save("modified_image_1.3.6.1.4.1.14519.5.2.1.6279.6001.202709423777326615340853838834.png")

In [89]:
def preprosess_color_of_image(img_path: str, blackining:bool): 
    image= Image.open(img_path)
    # Get the image's metadata
    image_info = image.info

   
    pixel_data = list(image.getdata())
    #To find out if we're dealing with a 16 bit or a 8 bit image:
    information = []
    for pixel in pixel_data:
        information.append(pixel)
    information.sort()
    if information[-1]>255: 
        bit = 16
    else: 
        bit = 8

    modified_pixel_data = []
    for pixel in pixel_data:
        intensity = pixel
        if blackining:
            if bit == 16: 
                if intensity <= 33200:
                    intensity = 0
                elif intensity >= 34500: 
                    intensity = 65535
            if bit == 8: 
                print("8-bit blir brukt")
                if intensity <= 129:
                    intensity = 0
                #elif intensity >= 135: 
                #    intensity = 255 
        modified_pixel_data.append(intensity)
    
    #########################################
    #Connected to the similar marked out region in the cell above 
    interval_list = n_colorInRow(modified_pixel_data,25)
    for interval in interval_list:
        for i in range(interval[0], interval[1]+1):
            modified_pixel_data[i] = 65535
    #########################################

    #############
    #Can try to highlight the spots inside the tumor (make the tumors white)
    #############
    
    image.putdata(modified_pixel_data)
    image.save(img_path)

In [90]:
train_val_test_img_path = ["/home/heris/data/RAW/YOLO_dataset/train/images/","/home/heris/data/RAW/YOLO_dataset/validation/images/","/home/heris/data/RAW/YOLO_dataset/test/images/"]
for path in train_val_test_img_path:
    for root, dirs, files in os.walk(path):
        if len(dirs) == 0: 
            for file in files: 
                preprosess_color_of_image(f'{root}{file}', True)